In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras

In [2]:
# 데이터 로드
data = pd.read_csv('./data/Regression_data.csv')

# One-hot encode the 'Sex' column
ohe = OneHotEncoder(sparse=False)
sex_encoded = ohe.fit_transform(data[['Sex']])
sex_encoded_df = pd.DataFrame(sex_encoded, columns=ohe.categories_[0])

# Concatenate the one-hot encoded columns to the original data frame
data_encoded = pd.concat([data.drop('Sex', axis=1), sex_encoded_df], axis=1)

# Separate the features from the target
X = data_encoded.drop('Rings', axis=1)
y = data_encoded['Rings']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create TensorFlow datasets for training and validation
train_ds = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
valid_ds = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))

# Cache the datasets and batch them
train_ds = train_ds.cache().shuffle(3500).batch(32)
valid_ds = valid_ds.cache().shuffle(1000).batch(32)



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:

# Define the L1 regularizer
l1 = tf.keras.regularizers.l1(1e-4)

# Define the model
nn = keras.models.Sequential([
    keras.layers.Dense(64, input_shape=[X_train.shape[1],], kernel_regularizer=l1),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Dense(32, kernel_regularizer=l1),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Dense(16, kernel_regularizer=l1),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Dense(8, kernel_regularizer=l1),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Dense(1),
])

# Compile the model
nn.compile(optimizer=keras.optimizers.Adam(0.01), loss='mse')

# Define the callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, verbose=0, mode='auto')
e_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True)

# Train the model
hist = nn.fit(train_ds, validation_data=valid_ds, epochs=200, callbacks=[reduce_lr, e_stop], verbose=0)


# # Plot the loss
# plt.plot(hist.history['loss'], label='train')
# plt.plot(hist.history['val_loss'], label='val')
# plt.xlabel('epoch')
# plt.ylabel('loss')
# plt.legend()
# plt.show()

In [ ]:
# # Save the model
# nn.save('my_model.h5')

In [6]:
from tensorflow.keras.models import load_model

# Load the model
nn = load_model('best_model_r2.h5')

In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Use the model to make predictions on the test set
y_pred = nn.predict(X_test)

# Flatten the arrays (because the output of the model is a 2D array)
y_test_flat = y_test.values.flatten()
y_pred_flat = y_pred.flatten()

# Calculate the performance metrics
mae = mean_absolute_error(y_test_flat, y_pred_flat)
mse = mean_squared_error(y_test_flat, y_pred_flat)
rmse = np.sqrt(mse)  # or mse**0.5
r2 = r2_score(y_test_flat, y_pred_flat)

# Print the performance metrics
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")


27/27 [==============================] - 0s 3ms/step
MAE: 1.4418711713626624
MSE: 4.2280861108604055
RMSE: 2.0562310451066548
R2 Score: 0.609422256652145


In [8]:
# Predict on the test data
y_pred = nn.predict(X_test).flatten()

# Calculate the MAPE
mape = np.mean(np.abs((y_test - y_pred) / y_test))

# Calculate 1 - MAPE
accuracy = 1 - mape

accuracy


27/27 [==============================] - 0s 2ms/step


0.8558894472508325

In [13]:
from tensorflow.keras.models import load_model

# Load the model
nn = load_model('best_model_acc.h5')

In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Use the model to make predictions on the test set
y_pred = nn.predict(X_test)

# Flatten the arrays (because the output of the model is a 2D array)
y_test_flat = y_test.values.flatten()
y_pred_flat = y_pred.flatten()

# Calculate the performance metrics
mae = mean_absolute_error(y_test_flat, y_pred_flat)
mse = mean_squared_error(y_test_flat, y_pred_flat)
rmse = np.sqrt(mse)  # or mse**0.5
r2 = r2_score(y_test_flat, y_pred_flat)

# Print the performance metrics
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")


27/27 [==============================] - 0s 3ms/step
MAE: 1.430631273100821
MSE: 4.378940607137779
RMSE: 2.092591839594568
R2 Score: 0.5954867768191894


In [15]:
# Predict on the test data
y_pred = nn.predict(X_test).flatten()

# Calculate the MAPE
mape = np.mean(np.abs((y_test - y_pred) / y_test))

# Calculate 1 - MAPE
accuracy = 1 - mape

accuracy


27/27 [==============================] - 0s 2ms/step


0.8624262486964518

### optuna 를 해보자

In [ ]:
global nn

def reg_nn_objective(trial, X_train, X_test, y_train, y_test):
    global nn
    # Define the L1 regularizer
    l1 = tf.keras.regularizers.l1(trial.suggest_float("l1", 1e-5, 1e-1, log=True))

    # Define the model
    nn = keras.models.Sequential()
    nn.add(keras.layers.Dense(trial.suggest_int("n_units1", 4, 64), input_shape=[X_train.shape[1],], kernel_regularizer=l1))
    nn.add(keras.layers.BatchNormalization())
    nn.add(keras.layers.ReLU())
    nn.add(keras.layers.Dense(trial.suggest_int("n_units2", 4, 64), kernel_regularizer=l1))
    nn.add(keras.layers.BatchNormalization())
    nn.add(keras.layers.ReLU())
    nn.add(keras.layers.Dense(trial.suggest_int("n_units3", 4, 64), kernel_regularizer=l1))
    nn.add(keras.layers.BatchNormalization())
    nn.add(keras.layers.ReLU())
    nn.add(keras.layers.Dense(trial.suggest_int("n_units4", 4, 64), kernel_regularizer=l1))
    nn.add(keras.layers.BatchNormalization())
    nn.add(keras.layers.ReLU())
    nn.add(keras.layers.Dense(1))

    # Compile the model
    nn.compile(optimizer=keras.optimizers.Adam(trial.suggest_float("lr", 1e-4, 1e-2, log=True)), loss='mse')

    # Create TensorFlow datasets for training and validation
    train_ds = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
    valid_ds = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))

    # Cache the datasets and batch them
    train_ds = train_ds.cache().shuffle(3500).batch(32)
    valid_ds = valid_ds.cache().shuffle(1000).batch(32)

    # Define the callbacks
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, verbose=0, mode='auto')
    e_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True)

    # Train the model
    hist = nn.fit(train_ds, validation_data=valid_ds, epochs=200, callbacks=[reduce_lr, e_stop], verbose=0)

    # Calculate the R^2 score
    y_pred = nn.predict(X_test)
    r2 = r2_score(y_test, y_pred)

    # Return R^2 to maximize
    return r2

def save_model_callback(study, trial):
    global nn
    if study.best_trial.number == trial.number:
        nn.save('best_model_r2.h5')  # Save the model

# Create the study
reg_nn_study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler()
)

# Optimize the study
reg_nn_study.optimize(
    lambda trial: reg_nn_objective(trial, X_train, X_test, y_train, y_test),
    n_trials=500, callbacks=[save_model_callback]
)


[I 2023-07-30 12:07:48,369] A new study created in memory with name: no-name-372aa96d-1e22-4487-80a9-e8db5258210a
Exception ignored in: <function UniquePtr.__del__ at 0x7869341ba830>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/c_api_util.py", line 74, in __del__
    self.deleter(obj)
KeyboardInterrupt: 
